In [1]:
### Benchmarks for the 'generate_moves' function

In [ ]:
from guard_towers import Board, BOARD_SIZE, Piece
from time import time
import pandas as pd

In [9]:
from timeit import timeit
from guard_towers import Board, Piece, BOARD_SIZE   # adjust import
from typing import Dict

def empty_board() -> Board:
    """Start with a completely blank board (no pieces)."""
    b = Board()
    for y in range(BOARD_SIZE):
        for x in range(BOARD_SIZE):
            b.grid[y][x] = None
    return b

def board_from_dict(pieces: Dict[str, Piece]) -> Board:
    """Create a board from a {square: Piece(...)} mapping."""
    b = empty_board()
    for sq, pc in pieces.items():
        b.place(sq, pc)
    return b

def time_generate_moves(board: Board, player: str, repeats: int = 10_000) -> float:
    """Return time (seconds) to call generate_moves `repeats` times."""
    return timeit(lambda: board.generate_moves(player), number=repeats)


In [ ]:
initial_board = Board()
print("Blue to move:", time_generate_moves(initial_board, 'b'))
print("Red  to move:", time_generate_moves(initial_board, 'r'))


Blue to move: 0.1319388750125654
Red  to move: 0.10897666699020192


In [11]:
mid_game_pieces = {
    # Blue
    'D3': Piece('b', 'guard'),
    'E3': Piece('b', 'tower', 3),
    'B4': Piece('b', 'tower', 2),
    'G1': Piece('b', 'tower', 1),

    # Red
    'D6': Piece('r', 'guard'),
    'C5': Piece('r', 'tower', 2),
    'F6': Piece('r', 'tower', 4),
    'A7': Piece('r', 'tower', 1),
}
mid_game_board = board_from_dict(mid_game_pieces)

print("Blue to move:", time_generate_moves(mid_game_board, 'b'))
print("Red  to move:", time_generate_moves(mid_game_board, 'r'))


Blue to move: 0.13726724998559803
Red  to move: 0.10538349999114871


In [12]:
end_game_pieces = {
    # Blue
    'D2': Piece('b', 'guard'),
    'D3': Piece('b', 'tower', 4),

    # Red
    'D7': Piece('r', 'guard'),
    'D5': Piece('r', 'tower', 3),
}
end_game_board = board_from_dict(end_game_pieces)

print("Blue to move:", time_generate_moves(end_game_board, 'b'))
print("Red  to move:", time_generate_moves(end_game_board, 'r'))


Blue to move: 0.09080900001572445
Red  to move: 0.0646641249768436


### Benchmark eval function

In [2]:
import time
from guard_towers import fen_to_board, Board
from evaluation import evaluate

def benchmark_evaluate(fen: str, player: str, iterations: int = 10_000):
    board, player = fen_to_board(fen)

    # Time the evaluation function over multiple iterations
    start = time.time()
    for _ in range(iterations):
        evaluate(board, player)
    end = time.time()

    total_time = end - start
    avg_time = total_time / iterations
    print(f"Evaluated {iterations} times in {total_time:.4f} seconds.")
    print(f"Average time per call: {avg_time:.8f} seconds.")


# print how many moves are possible
def count_moves(board: Board, player: str) -> int:
    """Count the number of possible moves for a player."""
    moves = board.generate_moves(player)
    return len(moves)

fen = 'r3RG5/r16/b16/7/7/7/3BG3 r'
benchmark_evaluate(fen, player='r')
board, player = fen_to_board(fen)
num_moves = count_moves(board, player)
print(f"Number of possible moves for {player}: {num_moves}")

fen = 'b76/3RG3/7/7/7/7/3BG3 b'
benchmark_evaluate(fen, player='b')
board, player = fen_to_board(fen)
num_moves = count_moves(board, player)
print(f"Number of possible moves for {player}: {num_moves}")

Evaluated 10000 times in 0.0292 seconds.
Average time per call: 0.00000292 seconds.
Number of possible moves for r: 6
Evaluated 10000 times in 0.0215 seconds.
Average time per call: 0.00000215 seconds.
Number of possible moves for b: 15


# Alpha-Beta, with TT and pruning

In [1]:
import time
from guard_towers import fen_to_board
from evaluation import find_best_move, transposition_table, pv_table, depth_move_counters

def benchmark_find_best_move_avg(fen: str, player: str, depths: list, runs_per_depth: int = 10):
    board_init, _ = fen_to_board(fen)

    for depth in depths:
        total_time = 0.0
        last_move = None
        for _ in range(runs_per_depth):
            board = board_init.copy()

            transposition_table.clear()
            pv_table.clear()
            
            start = time.perf_counter()
            move = find_best_move(board, player, base_depth=depth)
            end = time.perf_counter()
            total_time += (end - start)
            last_move = move
        avg_time = total_time / runs_per_depth
        print(f"Depth {depth}: Avg time = {avg_time:.4f}s over {runs_per_depth} runs — Last move: {last_move}")

depth_move_counters.clear()
benchmark_find_best_move_avg('r3RG5/r16/b16/7/7/7/3BG3 r', player='r', depths=[1, 2, 3, 4, 5, 6])
last_key = max(depth_move_counters.keys())
print(depth_move_counters[last_key].items())
# print(depth_move_counters)
depth_move_counters.clear()
benchmark_find_best_move_avg('b76/3RG3/7/7/7/7/3BG3 b', player='b', depths=[1, 2, 3, 4, 5, 6])
last_key = max(depth_move_counters.keys())
print(depth_move_counters[last_key].items())
# print(depth_move_counters)

Depth 1: Avg time = 0.0004s over 10 runs — Last move: A6-A7-1
Depth 2: Avg time = 0.0016s over 10 runs — Last move: A6-A5-1
Depth 3: Avg time = 0.0052s over 10 runs — Last move: A6-A5-1
Depth 4: Avg time = 0.0100s over 10 runs — Last move: A6-A5-1
Depth 5: Avg time = 0.0188s over 10 runs — Last move: A6-A5-1
Depth 6: Avg time = 0.0353s over 10 runs — Last move: A6-A5-1
dict_items([(1, 6), (2, 38), (3, 110), (4, 285), (5, 1253), (6, 1638), (7, 2846)])
Depth 1: Avg time = 0.0006s over 10 runs — Last move: A7-D7-3
Depth 2: Avg time = 0.0015s over 10 runs — Last move: A7-D7-3
Depth 3: Avg time = 0.0026s over 10 runs — Last move: A7-D7-3
Depth 4: Avg time = 0.0177s over 10 runs — Last move: A7-D7-3
Depth 5: Avg time = 0.0194s over 10 runs — Last move: A7-D7-3
Depth 6: Avg time = 0.3540s over 10 runs — Last move: A7-D7-3
dict_items([(1, 15), (2, 64), (3, 587), (4, 1335), (5, 12060), (6, 18736), (7, 97591)])


# Mini-max

In [2]:
import time
from guard_towers import fen_to_board
from evaluation import find_best_move_MM, transposition_table, pv_table, depth_move_counters

def benchmark_find_best_move_MM_avg(fen: str, player: str, depths: list, runs_per_depth: int = 10):
    board_init, _ = fen_to_board(fen)

    for depth in depths:
        total_time = 0.0
        last_move = None
        for _ in range(runs_per_depth):
            board = board_init.copy()

            transposition_table.clear()
            pv_table.clear()
            
            start = time.perf_counter()
            move = find_best_move_MM(board, player, base_depth=depth)
            end = time.perf_counter()
            total_time += (end - start)
            last_move = move
        avg_time = total_time / runs_per_depth
        print(f"Depth {depth}: Avg time = {avg_time:.4f}s over {runs_per_depth} runs — Last move: {last_move}")

depth_move_counters.clear()
benchmark_find_best_move_MM_avg('r3RG5/r16/b16/7/7/7/3BG3 r', player='r', depths=[1, 2, 3, 4, 5, 6])
last_key = max(depth_move_counters.keys())
print(depth_move_counters[last_key].items())
# print(depth_move_counters)
depth_move_counters.clear()
benchmark_find_best_move_MM_avg('b76/3RG3/7/7/7/7/3BG3 b', player='b', depths=[1, 2, 3, 4, 5, 6])
last_key = max(depth_move_counters.keys())
print(depth_move_counters[last_key].items())
# print(depth_move_counters)

Depth 1: Avg time = 0.0002s over 10 runs — Last move: A6-A5-1
Depth 2: Avg time = 0.0010s over 10 runs — Last move: A6-A5-1
Depth 3: Avg time = 0.0036s over 10 runs — Last move: A6-A5-1
Depth 4: Avg time = 0.0214s over 10 runs — Last move: A6-A5-1
Depth 5: Avg time = 0.1571s over 10 runs — Last move: A6-A5-1
Depth 6: Avg time = 1.1083s over 10 runs — Last move: A6-A5-1
dict_items([(1, 6), (2, 32), (3, 219), (4, 1218), (5, 10859), (6, 62771), (7, 632519)])
Depth 1: Avg time = 0.0002s over 10 runs — Last move: A7-A1-6
Depth 2: Avg time = 0.0013s over 10 runs — Last move: A7-A1-6
Depth 3: Avg time = 0.0120s over 10 runs — Last move: A7-A1-6
Depth 4: Avg time = 0.0752s over 10 runs — Last move: A7-A1-6
Depth 5: Avg time = 0.7312s over 10 runs — Last move: A7-A1-6
Depth 6: Avg time = 4.8247s over 10 runs — Last move: A7-A1-6
dict_items([(1, 15), (2, 60), (3, 945), (4, 3517), (5, 59634), (6, 220917), (7, 3984485)])


### MCTS Test

In [1]:
import time
from guard_towers import fen_to_board
from evaluation import mcts_find_best_move

def benchmark_find_best_move_MCTS_avg(fen: str, player: str, sim_counts: list, runs_per_sim: int = 10):
    board_init, _ = fen_to_board(fen)

    for sims in sim_counts:
        total_time = 0.0
        last_move = None
        for _ in range(runs_per_sim):
            board = board_init.copy()

            start = time.perf_counter()
            move = mcts_find_best_move(board, player, simulations=sims)
            end = time.perf_counter()
            total_time += (end - start)
            last_move = move
        avg_time = total_time / runs_per_sim
        print(f"Simulations {sims}: Avg time = {avg_time:.4f}s over {runs_per_sim} runs — Last move: {last_move}")

# Example usage:
benchmark_find_best_move_MCTS_avg('r3RG5/r16/b16/7/7/7/3BG3 r', player='r', sim_counts=[10, 50, 100, 200, 500, 1000])
benchmark_find_best_move_MCTS_avg('b76/3RG3/7/7/7/7/3BG3 b', player='b', sim_counts=[10, 50, 100, 200, 500, 1000])

Simulations 10: Avg time = 0.0027s over 10 runs — Last move: A6-A5-1
Simulations 50: Avg time = 0.0207s over 10 runs — Last move: A6-A5-1
Simulations 100: Avg time = 0.0210s over 10 runs — Last move: A6-A5-1
Simulations 200: Avg time = 0.0427s over 10 runs — Last move: A6-A5-1
Simulations 500: Avg time = 0.0941s over 10 runs — Last move: A6-A5-1
Simulations 1000: Avg time = 0.1872s over 10 runs — Last move: A6-A5-1
Simulations 10: Avg time = 0.0019s over 10 runs — Last move: D1-E1-1
Simulations 50: Avg time = 0.0093s over 10 runs — Last move: A7-A6-1
Simulations 100: Avg time = 0.0189s over 10 runs — Last move: A7-F7-5
Simulations 200: Avg time = 0.0282s over 10 runs — Last move: D1-E1-1
Simulations 500: Avg time = 0.0320s over 10 runs — Last move: A7-A6-1
Simulations 1000: Avg time = 0.0694s over 10 runs — Last move: A7-D7-3
